In [1]:

import pandas as pd
import numpy as np
import math
from collections import Counter

In [2]:
sensor_data = pd.DataFrame.from_csv("../../data/database/sensor_data.csv")
Counter(sensor_data.TagName)

Counter({'Start1': 213100,
         'Start2': 213808,
         'Tag0': 352461,
         'Tag1': 272234,
         'Tag2': 377420,
         'Tag3': 398370,
         'Tag4': 428642,
         'Tag5': 435086,
         'Tag6': 358288,
         'Tag7': 315266,
         'Tag8': 431390,
         'Tag9': 381880})

In [3]:
sensor_data_acc = sensor_data[sensor_data.SENSORTYPE == 1]
sensor_data_acc_number_tag = sensor_data_acc[~((sensor_data_acc.TagName == 'Start1') | (sensor_data_acc.TagName == 'Start2'))]

In [4]:
Counter(sensor_data_acc_number_tag.TagName)

Counter({'Tag0': 53436,
         'Tag1': 41185,
         'Tag2': 57227,
         'Tag3': 60499,
         'Tag4': 65089,
         'Tag5': 66044,
         'Tag6': 54351,
         'Tag7': 47811,
         'Tag8': 65394,
         'Tag9': 57911})

In [5]:
# tag_id_groupby = sensor_data_acc_tag12.groupby(['TagName', 'tester_id'])
tag_id_groupby = sensor_data_acc_number_tag.groupby(['TagName', 'tester_id'])
tag_id_dict = tag_id_groupby.groups
y = list(tag_id_dict.keys())
y = [i[0] for i in y]
y = np.array(y)

In [6]:
N_frame_no = 1 

test_percent = 0.1 # 10% samples are used for testing

In [7]:
def shrink_array(array,size):
    
    ratio = float(len(array)) / float(size)
    res = []
    for i in range(size):
        res.append(np.mean(array[math.floor(i*ratio):math.ceil((i+1.0)*ratio)], axis = 0))
    return np.array(res)

In [8]:
def gesture_features(accs):
    if N_frame_no > 1:
        Ls = math.floor(len(accs)/ (N_frame_no + 1))
        segments = None
        for i in range(N_frame_no + 1):
            if segments is None:
                segments = np.array([accs[i*Ls:(i+1)*Ls]])
            else:
                segments = np.append(segments, np.array([accs[i*Ls:(i+1)*Ls]]), axis=0)

        frames = None
        for i in range(N_frame_no):
            cur_frame = segments[i:i+2]
            cur_frame = cur_frame.reshape((cur_frame.shape[0]*cur_frame.shape[1],cur_frame.shape[2]))
            if frames is None:
                frames = np.array([cur_frame])
            else:
                frames = np.append(frames, np.array([cur_frame]), axis = 0)
        return np.array([frame_features(f) for f in frames]).reshape(-1)
    else:
        return frame_features(accs).reshape(-1)

In [9]:
def frame_features(cur_frame):
    
    mean_cur_frame = np.mean(cur_frame ,axis= 0)
    
    energy_cur_frame=[]
    for T in range(cur_frame.shape[1]): #x,y,z
        T_sum = 0
        for i in cur_frame[:,T]:
            T_sum += math.pow(abs(i),2)
        energy_cur_frame.append(T_sum / len(cur_frame))
    energy_cur_frame = np.array(energy_cur_frame)
    
#     entropy_cur_frame = []
#     for T in range(cur_frame.shape[1]): #x,y,z
#         T_sum = 0
#         entropy_sum = 0
#         for i in cur_frame[:,T]:
#             T_sum += abs(i)
            
#         for m in cur_frame[:,T]:
#             p_m_T = abs(m)/T_sum
#             entropy_sum += p_m_T*math.log(1/p_m_T)

#         entropy_cur_frame.append(entropy_sum)
#     entropy_cur_frame = np.array(entropy_cur_frame)
    
    std_cur_frame = np.std(cur_frame, axis=0)
    
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    # correlation
    
    return np.concatenate((mean_cur_frame,energy_cur_frame,std_cur_frame))

In [10]:
X = []

for key in list(tag_id_dict.keys()):
    gesture_feature = gesture_features(tag_id_groupby.get_group(key)[['VALUES1', 'VALUES2', 'VALUES3']].values).reshape(-1)
    if np.all(~np.isnan(gesture_feature)):
        X.append(gesture_feature)
    else:
        print(key)
X = np.array(X)

# X = []

# for key in list(tag_id_dict.keys()):
#     gesture_feature = shrink_array(tag_id_groupby.get_group(key)[['VALUES1', 'VALUES2', 'VALUES3']].values, 200).reshape(-1)
#     if np.all(~np.isnan(gesture_feature)):
#         X.append(gesture_feature)
#     else:
#         print(key)
# X = np.array(X)

In [11]:
X.shape

(1070, 9)

In [12]:
from sklearn.svm import SVC
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [13]:
idx_list = list(range(len(X)))
np.random.shuffle(idx_list)
train_idx = idx_list[:math.floor(len(X) * (1- test_percent))]
test_idx = idx_list[math.floor(len(X) * (1- test_percent)):]

In [ ]:
# y[y == 'Tag2'] = 2
# y[y == 'Tag1'] = 1

train_x = X[train_idx]
train_y = y[train_idx]
test_x = X[test_idx]
test_y = y[test_idx]

In [ ]:
clf4 = SVC(kernel='poly', C=1, degree=3, verbose = True)
clf4.fit(train_x, train_y) 
# joblib.dump(clf4, '../../Results/baseline SVC degree4.') 
print(classification_report(test_y, clf4.predict(test_x)))


[LibSVM]